<a href="https://colab.research.google.com/github/gilsonauerswald/Bioinformatic_Projects/blob/main/Lesson_5_Intro_Molecular_Modeling_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 Introduction to Python for Molecular Modeling
Learning Objectives

*   Set up a molecular modeling environment in Google Colab
*   Retrieve protein structures from the PDB
*   Analyze protein sequences and structural geometry
*   Identify and analyze ligands

#Environment Setup

In [ ]:
# Install only what’s needed, avoid full notebook upgrade

!pip install "numpy<2.0" --quiet
!pip install biopython nglview==2.7.7 rdkit-pypi --quiet
!jupyter-nbextension enable nglview --py --sys-prefix

from google.colab import output
output.enable_custom_widget_manager()

# Confirm success
print("✅ Environment setup complete. Ready to import libraries!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.0 MB/s eta 0:00:00
Enabling notebook extension nglview-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Pa

# Step 1 & 2: Downloading & Parsing the PDB file from RCSB

In [ ]:
import nglview
import requests
from Bio.PDB import PDBParser
url = "https://files.rcsb.org/download/1AKE.pdb"
pdb_filename = "1AKE.pdb"

with open(pdb_filename, 'w') as f:
    f.write(requests.get(url).text)

# Step 2: Parse the structure
parser = PDBParser()
structure = parser.get_structure("1AKE", pdb_filename)

/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3794.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3858.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3915.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4156.
  warnings.warn(


# 🧬 Step 3: Analyze Protein Sequence


In [ ]:
residue_names = []
for model in structure:
    for chain in model:
        print(f"Chain ID: {chain.id}")
        for residue in chain:
            residue_names.append(residue.resname)
        print(f"Residue count: {len(residue_names)}")
        break  # one chain for preview
    break


Chain ID: A
Residue count: 456


# Step 4: Extract Amino Acid Sequence (One Chain)

In [ ]:
from Bio.SeqUtils import seq1

# Extract sequence from Chain A
chain_id = "A"
sequence = ""

for model in structure:
    for chain in model:
        if chain.id == chain_id:
            for residue in chain:
                if residue.get_id()[0] == " ":  # Skip heteroatoms/waters
                    try:
                        sequence += seq1(residue.resname)
                    except Exception:
                        sequence += "X"
            break

print(f"Amino acid sequence of chain {chain_id}:\n{sequence}")
print(f"Length: {len(sequence)}")

Amino acid sequence of chain A:
MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG
Length: 214


# ⚗️ Step 5: Ligand Molecular Weight with RDKit

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(aspirin_smiles)
mol_weight = Descriptors.MolWt(mol)

print(f"Molecular weight of Aspirin: {mol_weight:.2f} Da")


Molecular weight of Aspirin: 180.16 Da


# Step 6: Visualize with NGLView

In [ ]:
# @title Default title text
import nglview as nv

view = nv.show_biopython(structure)
view

NGLWidget()

# Step 6: Identify Ligands in the Structure

In [ ]:
from Bio.PDB.Polypeptide import is_aa

ligands = []
for chain in model:
    for res in chain:
        # Check if the residue is not an amino acid and not water
        if not is_aa(res) and res.get_resname() != 'HOH':
            ligands.append(res)

print(f"Found {len(ligands)} ligand(s).")
for lig in ligands:
    print(f"{lig.get_resname()} in Chain {lig.get_parent().id}, Residue {lig.id[1]}")


Found 2 ligand(s).
AP5 in Chain A, Residue 215
AP5 in Chain B, Residue 215


In [ ]:
import numpy as np

# Initialize an empty list to store ligand atoms
ligand_atoms = []
# Get all atoms from the identified ligands
for lig in ligands:
    for atom in lig.get_atoms():
        ligand_atoms.append(atom)

# Get centroid of ligand atoms
ligand_coords = np.array([atom.coord for atom in ligand_atoms])
ligand_centroid = ligand_coords.mean(axis=0)

# Get residues with CA atom within 5 Å of ligand centroid
binding_site_residues = set()

for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_id()[0] == " ":
                if 'CA' in residue:
                    ca_coord = residue['CA'].coord
                    distance = np.linalg.norm(ca_coord - ligand_centroid)
                    if distance <= 5.0:
                        binding_site_residues.add(residue)

print(f"Binding site residues within 4 Å of ligand centroid: {len(binding_site_residues)}")
for res in binding_site_residues:
    print(f"Chain {res.get_parent().id} | {res.resname} {res.id[1]}")


Binding site residues within 4 Å of ligand centroid: 4
Chain B | GLU 151
Chain B | ASP 146
Chain B | THR 149
Chain B | GLY 150
